In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [2]:
data_age = pd.read_csv('data/cleaned_age.csv')
data_health = pd.read_csv('data/cleaned_health.csv')
data_heat = pd.read_csv('data/cleaned_heat.csv')
data_depr = pd.read_csv('data/cleaned_depr.csv')
data_hhold = pd.read_csv('data/cleaned_hhold.csv')
data_ethgrp = pd.read_csv('data/cleaned_ethgrp.csv')
data_cars = pd.read_csv('data/cleaned_cars.csv')

meta_data = pd.read_csv('data/meta_all.csv')

lsoa_data = pd.read_csv('manchester_data_la.csv', sep = ',')

#data_ge = pd.read_csv('raw_data/man_meanGandE.csv', sep = ',')

In [3]:
def prop_of_tot(old_data):
    
    data = old_data.set_index('GEO_CODE').astype(int)
    data['sum'] = data.sum(axis = 1, skipna = all)
    
    new_df = pd.DataFrame(index = data.index, columns = data.columns)
    for i in range(len(data)):
        for j in range(int(len(data.columns) - 1)):
            new_df.iloc[i][j] = (data.iloc[i][j] / data.iloc[i][int(len(data.columns) - 1)])
    
    new_df = new_df.dropna(axis = 1)

    return(new_df)

In [4]:
prop_health = prop_of_tot(data_health)
prop_heat = prop_of_tot(data_heat)
prop_depr = prop_of_tot(data_depr)
prop_cars = prop_of_tot(data_cars)
prop_ethgrp = prop_of_tot(data_ethgrp)

In [5]:
prop_health['badhealth'] = prop_health['F1777'] + prop_health['F1782']

In [6]:
prop_depr['depr_3+'] = prop_depr['F1000'] + prop_depr['F1001']

In [7]:
prop_ethgrp['F166x'] = prop_ethgrp['F1664'] + prop_ethgrp['F1665'] + prop_ethgrp['F1666'] + prop_ethgrp['F1667']

In [8]:
data_ethgrp = data_ethgrp.set_index('GEO_CODE')

for i in range(len(data_ethgrp)):
    for j in range(len(data_ethgrp.columns)):
        data_ethgrp.iloc[i][j] = data_ethgrp.iloc[i][j] ** 2

data_ethgrp['hhi'] = data_ethgrp.sum(axis = 1, skipna = all)

In [9]:
data_ethgrp = data_ethgrp[['hhi']]

In [11]:
# normalise hhi --> turns to array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(data_ethgrp.values)

C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [12]:
hhi_list = []

for i in range(len(x_scaled)):
    hhi_list.append(float(x_scaled[i]))

In [13]:
data_ethgrp['hhi_norm'] = hhi_list

In [14]:
data_ethgrp = data_ethgrp[['hhi_norm']].join(prop_ethgrp[['F166x']])

In [15]:
data_hhold = data_hhold.set_index('GEO_CODE')
data_age = data_age.set_index('GEO_CODE')

In [16]:
prop_hhold = pd.DataFrame(index = data_hhold.index, columns = data_hhold.columns)

for i in range(len(data_hhold)):
    for j in range(1, len(data_hhold.columns)):
        prop_hhold.iloc[i][j] = (data_hhold.iloc[i][j] / data_hhold.iloc[i][0])
        
prop_hhold = prop_hhold.dropna(axis = 1)

In [36]:
# join data

man_data = prop_health.join(prop_heat).join(prop_depr).join(prop_cars).join(prop_hhold).join(data_age).join(data_ethgrp)

In [37]:
tokeep = ['F184', 'F185', 'badhealth', 'F2345', 'F997', 'depr_3+', 'F1838', 'F991', 'hhi_norm', 'F166x'] 

man_data = (man_data[tokeep].rename(columns = {'F184':'age_mean',
                                               'F185':'age_median',
                                               'F2345':'centheat',
                                               'F997':'nodepr',
                                               'F1838':'studenthhold',
                                               'F166x':'ethwhite',
                                               'F991':'nocars',
                                               'hhi_norm':'eth_hhi'})\
           .astype(np.float))

In [19]:
#data_ge = data_ge.set_index('geo_code')[['mean_elect', 'mean_gas']]

In [20]:
lsoa_data = lsoa_data.set_index('geo_code_lsoa')

In [38]:
all_data = lsoa_data[['local_auth']].join(man_data)#.join(data_ge)

In [22]:
all_data.head()

,local_auth,age_mean,badhealth,centheat,nodepr,depr_3+,studenthhold,nocars,eth_hhi,F166x,mean_elect,mean_gas
geo_code_lsoa,,,,,,,,,,,,
E01005268,Manchester,40.40783,0.054014,0.976861,0.509054,0.026157,0.001006,0.280684,0.285152,0.884539,3649.224138,14515.64286
E01033656,Manchester,22.29914,0.004585,0.909330,0.371879,0.003942,0.268068,0.674113,0.361815,0.593410,6636.307692,17387.00000
E01005184,Manchester,29.31636,0.038190,0.966624,0.441592,0.048780,0.219512,0.410783,0.159142,0.672324,3728.692308,12046.02857
E01005298,Manchester,30.76403,0.052861,0.973479,0.483619,0.060842,0.015601,0.260530,0.064515,0.407629,3827.666667,15935.56000
E01005300,Manchester,36.61435,0.065343,0.977551,0.404082,0.057143,0.004082,0.159184,0.060058,0.357463,4164.428571,19614.64286


In [ ]:
'''
all_data['centre_dummy'] = np.nan
all_data['centre'] = 'unknown'

for i in range(len(all_data)):
    if all_data['local_auth'][i] == 'Manchester':
        all_data['centre_dummy'][i] = int(1)
        all_data['centre'][i] = 'Yes'
    else:
        all_data['centre_dummy'][i] = int(0)
        all_data['centre'][i] = 'No'
'''

In [ ]:
'''all_data['depr_3+_dummy'] = np.nan
all_data['depr_3+_cat'] = 'unknown'

for i in range(len(all_data)):
    if all_data['depr_3+'][i] >= 0.1:
        all_data['depr_3+_dummy'][i] = int(1)
        all_data['depr_3+_cat'][i] = 'None'
    else:
        all_data['depr_3+_dummy'][i] = int(0)
        all_data['depr_3+_cat'][i] = '3+'
'''

In [ ]:
'''
(q1, q2, q3) = all_data['nodepr'].quantile([0.25,0.5,0.75])

all_data['nodepr_quar'] = 'unknown'

for i in range(len(all_data)):
    all_data['nodepr_quar'][i] = 'Q4'
    
    if all_data['nodepr'][i] <= q3:
        all_data['nodepr_quar'][i] = 'Q3'
        
    if all_data['nodepr'][i] <= q2:
        all_data['nodepr_quar'][i] = 'Q2'
    
    if all_data['nodepr'][i] <= q1:
        all_data['nodepr_quar'][i] = 'Q1'
'''        

In [39]:
all_data['nodepr_med'] = 'unknown'

for i in range(len(all_data)):
    if all_data['nodepr'][i] <= all_data['nodepr'].median():
        all_data['nodepr_med'][i] = 'below_med'
    else:
        all_data['nodepr_med'][i] = 'above_med'   

C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [ ]:
'''
(t1, t2) = all_data['nodepr'].quantile([0.33, 0.67])

all_data['nodepr_thir'] = 'unknown'

for i in range(len(all_data)):
    all_data['nodepr_thir'][i] = 'T3'
        
    if all_data['nodepr'][i] <= t2:
        all_data['nodepr_thir'][i] = 'T2'
    
    if all_data['nodepr'][i] <= t1:
        all_data['nodepr_thir'][i] = 'T1'
'''

In [ ]:
'''
(q1, q2, q3) = all_data['nocars'].quantile([0.25,0.5,0.75])

all_data['nocars_quar'] = 'unknown'

for i in range(len(all_data)):
    all_data['nocars_quar'][i] = 'Q4'
    
    if all_data['nocars'][i] <= q3:
        all_data['nocars_quar'][i] = 'Q3'
        
    if all_data['nocars'][i] <= q2:
        all_data['nocars_quar'][i] = 'Q2'
    
    if all_data['nocars'][i] <= q1:
        all_data['nocars_quar'][i] = 'Q1'
'''

In [40]:
all_data['nocars_med'] = 'unknown'

for i in range(len(all_data)):
    if all_data['nocars'][i] <= all_data['nocars'].median():
        all_data['nocars_med'][i] = 'below_med'
    else:
        all_data['nocars_med'][i] = 'above_med'

C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [43]:
(q1, q2, q3) = all_data['age_median'].quantile([0.25,0.5,0.75])

all_data['median_age_quar'] = 'unknown'

for i in range(len(all_data)):
    all_data['median_age_quar'][i] = 'Q4'
    
    if all_data['age_median'][i] <= q3:
        all_data['median_age_quar'][i] = 'Q3'
        
    if all_data['age_median'][i] <= q2:
        all_data['median_age_quar'][i] = 'Q2'
    
    if all_data['age_median'][i] <= q1:
        all_data['median_age_quar'][i] = 'Q1'

C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C

In [46]:
all_data['median_age_med'] = 'unknown'

for i in range(len(all_data)):
    if all_data['age_median'][i] <= all_data['age_median'].median():
        all_data['median_age_med'][i] = 'below'
        
    else:
        all_data['median_age_med'][i] = 'above'

C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\gylk\AppData\Local\Continuum\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
all_data.head()

,local_auth,age_mean,age_median,badhealth,centheat,nodepr,depr_3+,studenthhold,nocars,eth_hhi,ethwhite,nodepr_med,nocars_med,median_age_quar,median_age_med
geo_code_lsoa,,,,,,,,,,,,,,,
E01005268,Manchester,40.40783,36.0,0.054014,0.976861,0.509054,0.026157,0.001006,0.280684,0.285152,0.884539,above_med,above_med,below,below
E01033656,Manchester,22.29914,21.0,0.004585,0.909330,0.371879,0.003942,0.268068,0.674113,0.361815,0.593410,below_med,above_med,below,below
E01005184,Manchester,29.31636,22.0,0.038190,0.966624,0.441592,0.048780,0.219512,0.410783,0.159142,0.672324,above_med,above_med,below,below
E01005298,Manchester,30.76403,30.0,0.052861,0.973479,0.483619,0.060842,0.015601,0.260530,0.064515,0.407629,above_med,below_med,below,below
E01005300,Manchester,36.61435,34.0,0.065343,0.977551,0.404082,0.057143,0.004082,0.159184,0.060058,0.357463,above_med,below_med,below,below


In [48]:
all_data.to_csv('data/manchester_data.csv')